calculate new k factors

In [2]:
import shlex
import os
import time
import datetime
import logins
import subprocess
import numpy as np
from astropy.time import Time
import mysql.connector
from mysql.connector import Error

In [3]:
# give dates and calculate corresponding jd

start_year = 2021
start_month = 1
start_day = 1

end_year = 2023
end_month = 3
end_day = 31

start_date = str(start_year)+'-'+str(start_month)+'-'+str(start_day)+'T12:00:00.0'
end_date = str(end_year)+'-'+str(end_month)+'-'+str(end_day)+'T12:00:00.0'

times = [start_date, end_date]
t = Time(times, format='isot', scale='utc')
start_jd = t[0].jd
end_jd = t[1].jd
print(start_jd, end_jd)

2459216.0 2460035.0


In [4]:
#Add Database information
DB_HOST = 'alnilam.aavso.org' 
DB_USER = logins.sol_obs
DB_USER_PASSWORD = logins.sol_pass

DB_NAME = 'solar'
DB_Table = 'sunspot_counts'
#BACKUP_PATH = '/Users/aavso/solardump'

In [5]:
#Create Database Connection
try:
    connection = mysql.connector.connect(host=DB_HOST,
                                         database=DB_NAME,
                                         user=DB_USER,
                                         password=DB_USER_PASSWORD)
except Error as e:
    print("Error while connecting to MySQL", e)

mycursor = connection.cursor()

In [6]:
#find relevant observers
query = "SELECT obs, count(*) as count from sun_data where date >= "+str(start_jd)+ \
" and date <= "+str(end_jd)+" and revised = 0 group by obs having count >= 100 order by obs;"

In [7]:
mycursor.execute(query)

In [8]:
myresult = mycursor.fetchall()

In [9]:
observers = np.vstack(myresult)

In [10]:
#find ra values
query_ra = "SELECT count, jd from sunspot_counts where jd >= "+str(start_jd)+ \
" and jd <= "+str(end_jd)+"order by jd;"

In [11]:
mycursor.execute(query_ra)
myresult = mycursor.fetchall()
ra = np.vstack(myresult)

In [12]:
print(ra)

[[1.900000e+01 2.459216e+06]
 [8.000000e+00 2.459217e+06]
 [0.000000e+00 2.459218e+06]
 ...
 [1.080000e+02 2.460033e+06]
 [6.900000e+01 2.460034e+06]
 [5.200000e+01 2.460035e+06]]


In [13]:
obs = observers[0][0]
print(obs)
#get wolf numbers
query_wolf = 'SELECT W, date from sun_data where obs = "'+str(obs)+'" and date >= '+str(start_jd)+' \
and date <= '+str(end_jd)+' and revised = 0 order by date;'
print(query_wolf)

AAX
SELECT W, date from sun_data where obs = "AAX" and date >= 2459216.0 and date <= 2460035.0 and revised = 0 order by date;


In [14]:
query_k = "SELECT k from sun_obsconst where obs in"+str(tuple(list(observers[:,0])))+ \
"and revised = 0;"
mycursor.execute(query_k)
myresult = mycursor.fetchall()
k = np.vstack(myresult)


In [15]:
observers= np.column_stack((observers,k))

In [17]:
#calculate k factor
rows_n = []
for x in observers:
    obs = x[0]
    count = int(x[1])
    k = x[2]
    Ws = []
    ras = []
#    print(obs, count, k)
    query_wolf =query_wolf = 'SELECT W, date from sun_data where obs = "'+str(obs)+'" and date >= '+str(start_jd)+' \
and date <= '+str(end_jd)+' and revised = 0 order by date;'
#    print(query_wolf)
    mycursor.execute(query_wolf)
    myresult = mycursor.fetchall()
    wolf = np.vstack(myresult)
#    print(wolf)
#    quit()
    for row in wolf:
        W = int(row[0])
        date = int(row[1])
#        print(W, date)
        #calculate log of each wolf number
        if W != 0:
            logW = np.log10(W)
        else:
            logW = 0.1
        
        Ws.append(logW)
        #calculate log of each RA
        ra_new = ra[ra[:,1] == date][0][0]
        if ra_new != 0:
#            print(ra[ra[:,1] == date][0][0])
#            ra_new = ra[ra[:,1] == date][0][0]
            logra = np.log10(ra_new)
#            print("ra_new", ra_new)
        else: 
#            print("no ra")
            logra = 0.1
        ras.append(logra)
    logW_sum = sum(Ws)
    logRA_sum = sum(ras)
#    print(logW_sum, logRA_sum)
    logK = (logRA_sum-logW_sum)/count
    print("obscode", obs, "# observations", count)
    print("old k: ",k+"\n", "new k: ", round(10**logK,3))
    Kn = round(10**logK,3)
  #  print(round(10**logK,3))
    row_n = [obs, count, k, Kn]
    rows_n.append(row_n)

k_info = np.vstack((rows_n))

obscode AAX # observations 576
old k:  0.96
 new k:  0.76
obscode AJV # observations 464
old k:  0.946
 new k:  0.915
obscode ARAG # observations 798
old k:  1.083
 new k:  0.593
obscode ASA # observations 517
old k:  0.936
 new k:  0.889
obscode ATE # observations 675
old k:  0.0
 new k:  0.779
obscode BARH # observations 102
old k:  1.045
 new k:  1.336
obscode BATR # observations 225
old k:  0.875
 new k:  0.806
obscode BKL # observations 185
old k:  0.0
 new k:  1.001
obscode BMF # observations 582
old k:  0.0
 new k:  0.842
obscode BMIG # observations 393
old k:  0.0
 new k:  0.782
obscode BRAF # observations 182
old k:  0.0
 new k:  0.686
obscode BROB # observations 597
old k:  0.0
 new k:  0.848
obscode BXZ # observations 620
old k:  0.0
 new k:  0.96
obscode BZX # observations 674
old k:  0.798
 new k:  0.927
obscode CHAG # observations 130
old k:  0.898
 new k:  0.853
obscode CIOA # observations 159
old k:  0.628
 new k:  0.939
obscode CKB # observations 596
old k:  0.864
 new

In [23]:
header = "obscode"+" observations "+" old k "+" new k"
np.savetxt('K_factor_info.txt', k_info, header=header, fmt='%s')